In [1]:
!python -V

Python 3.12.3


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [23]:
import mlflow 

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc_taxi_experiment")

<Experiment: artifact_location='/workspaces/MLops/02-MLflow/mlruns/2', creation_time=1770796653345, experiment_id='2', last_update_time=1770796653345, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

In [24]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [25]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet')

In [26]:
len(df_train), len(df_val)

(54373, 51497)

In [27]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [28]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [29]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [33]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

5.995158501389459

In [32]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [37]:
#to keep track of the different runs with the help of mlflow

with mlflow.start_run():

    mlflow.set_tag("developer", "Sumanth") # helps in filtering out the model in later stages

    mlflow.log_param("train-data-path:","https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet")
    mlflow.log_param("valid-data-path:","https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet")
    alpha=0.01

    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

In [50]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [59]:
train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label = y_val)

In [60]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, 'validation')],
            early_stopping_rounds = 50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

        return{'loss': rmse, 'status': STATUS_OK}



In [61]:
search_space = {
    'max_depth' : scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate' : hp.loguniform('learning_rate', -3, 0), #exp(3), exp(1) --> [0.05, 1]
    'reg_alpha' : hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda' : hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight' : hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed':42
}

In [ ]:
best_result = fmin(
    fn = objective,
    space = search_space,
    algo = tpe.suggest,
    max_evals = 50,
    trials = Trials()
)

  0%|                                                                                                                                         | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/ept-tracking-env/lib/python3.12/site-packages/xgboost/callback.py:385: UserWarning: [12:13:05] WARNING: /__w/xgboost/xgboost/src/objective/regression_obj.cu:277: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:5.65841                                                                                                                                                          
[1]	validation-rmse:5.42861                                                                                                                                                          
[2]	validation-rmse:5.38775                                                                                                                                                          
[3]	validation-rmse:5.36420                                                                                                                                                          
[4]	validation-rmse:5.35324                                                                                                                                                          
[5]	validation-rmse:5.34195                                                               

/opt/conda/envs/ept-tracking-env/lib/python3.12/site-packages/xgboost/callback.py:385: UserWarning: [12:13:32] WARNING: /__w/xgboost/xgboost/src/objective/regression_obj.cu:277: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[3]	validation-rmse:5.65566                                                                                                                                                          
[4]	validation-rmse:5.54992                                                                                                                                                          
[5]	validation-rmse:5.49920                                                                                                                                                          
[6]	validation-rmse:5.47062                                                                                                                                                          
[7]	validation-rmse:5.45566                                                                                                                                                          
[8]	validation-rmse:5.44512                                                               

...

[0]	validation-rmse:8.04912                                                                                                                                                          
[1]	validation-rmse:7.25678                                                                                                                                                          
[2]	validation-rmse:6.67852                                                                                                                                                          
[3]	validation-rmse:6.26144                                                                                                                                                          
[4]	validation-rmse:5.96758                                                                                                                                                          
[5]	validation-rmse:5.76123                                                               

/opt/conda/envs/ept-tracking-env/lib/python3.12/site-packages/xgboost/callback.py:385: UserWarning: [12:23:05] WARNING: /__w/xgboost/xgboost/src/objective/regression_obj.cu:277: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:5.98492                                                                                                                                                          
[1]	validation-rmse:5.47693                                                                                                                                                          
[2]	validation-rmse:5.38834                                                                                                                                                          
[3]	validation-rmse:5.37567                                                                                                                                                          
[4]	validation-rmse:5.37042                                                                                                                                                          
[5]	validation-rmse:5.36667                                                               

/opt/conda/envs/ept-tracking-env/lib/python3.12/site-packages/xgboost/callback.py:385: UserWarning: [12:23:20] WARNING: /__w/xgboost/xgboost/src/objective/regression_obj.cu:277: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:7.75465                                                                                                                                                          
[3]	validation-rmse:7.41298                                                                                                                                                          
[4]	validation-rmse:7.11867                                                                                                                                                          
[5]	validation-rmse:6.86315                                                                                                                                                          
[6]	validation-rmse:6.64576                                                                                                                                                          
[7]	validation-rmse:6.45757                                                               